In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection, naive_bayes, svm
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
train_url = '/home/thinker/NLP/NLP/FromScratch/error detection/data/data/train_data.csv'
test_url = '/home/thinker/NLP/NLP/FromScratch/error detection/data/data/test_data.csv'
df_train = pd.read_csv(train_url)
df_test = pd.read_csv(test_url)
stop_words_df = pd.read_excel('/home/thinker/NLP/NLP/FromScratch/error detection/data/stopwords_bangla.xlsx',index_col=False)

In [3]:
STOPWORDS = set([word.strip() for word in stop_words_df['words']])

In [4]:
import re
def preprocess(x):
    html_pattern = re.compile('<.*?>')
    x = html_pattern.sub(r'', x)
    x = " ".join([word for word in str(x).split() if word not in STOPWORDS])
    return x
df_train['Comment'] = df_train['Comment'].apply(lambda x: preprocess(x))
df_test['Comment'] = df_test['Comment'].apply(lambda x:preprocess(x))

In [5]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(df_train.Error)
Test_Y = Encoder.fit_transform(df_test.Error)

In [6]:
df_all  = pd.concat([df_train, df_test], ignore_index=True)
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(df_all['Comment'])
Train_X_Tfidf = Tfidf_vect.transform(df_train['Comment'])
Test_X_Tfidf = Tfidf_vect.transform(df_test['Comment'])

In [7]:
model = XGBClassifier(
    n_estimators=100,  # Number of boosting rounds (trees)
    learning_rate=0.1,  # Step size shrinkage to prevent overfitting
    max_depth=3,        # Maximum depth of each tree
    subsample=0.8,      # Fraction of samples used for fitting each tree
    colsample_bytree=0.8,  # Fraction of features used for fitting each tree
    random_state=42
)

In [9]:
model.fit(Train_X_Tfidf, Train_Y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=42, ...)

In [10]:
y_pred = model.predict(Test_X_Tfidf)

In [11]:
print(metrics.classification_report(Test_Y, y_pred,digits = 4))

              precision    recall  f1-score   support

           0     0.6250    0.9340    0.7489      1167
           1     0.7116    0.2251    0.3420       844

    accuracy                         0.6365      2011
   macro avg     0.6683    0.5796    0.5455      2011
weighted avg     0.6613    0.6365    0.5781      2011

